In [1]:
import os
import json
import time
import glob
import shutil
import os.path

import pandas as pd
import numpy as np

from datetime import date, datetime

%load_ext lab_black

In [2]:
data_folder = "/home/gillard/Bureau/MISP_Analysis/Back-up/WEIS2022/Events_created"
output_folder = "/home/gillard/Bureau/MISP_Analysis/Back-up/WEIS2022"

In [3]:
events = []
attributes = []
related_events = []
tags = []
galaxies = []
galaxyclusters = []

for path, subdirs, files in os.walk(data_folder):
    for file in files:
        if file.startswith("event_") and file.endswith(".json"):
            with open(os.path.join(path, file), "r") as event_file:
                print(os.path.join(path, file), end="\r")
                event = json.loads("[" + event_file.read().replace("}{", "},{") + "]")[
                    -1
                ]["Event"]

            events.append(
                {
                    "Event ID": int(event["id"]),
                    "Orgc ID": int(event["orgc_id"]),
                    "Org ID": int(event["org_id"]),
                    "Org event creator": event["Orgc"]["uuid"],
                    "Org UUID": event["Org"]["uuid"],
                    "Date": event["date"],
                    "Threat Level ID": int(event["threat_level_id"]),
                    "Published": event["published"],
                    "UUID": event["uuid"],
                    "Attribute Count": int(event["attribute_count"]),
                    "Related Events Count": len(event["RelatedEvent"]),
                    "Analysis": event["analysis"],
                    "Timestamp": int(event["timestamp"]),
                    "Distribution": int(event["distribution"]),
                    "Published Timestamp": int(event["publish_timestamp"]),
                    "Info": event["info"],
                }
            )

            for attribute in event["Attribute"]:
                attributes.append(
                    {
                        "Event ID": int(attribute["event_id"]),
                        "Orgc ID": int(event["orgc_id"]),
                        "Org ID": int(event["org_id"]),
                        "Attribute ID": int(attribute["id"]),
                        "UUID": attribute["uuid"],
                        "Attribute Type": attribute["type"],
                        "Attribute Category": attribute["category"],
                        "Distribution": int(attribute["distribution"]),
                        "Timestamp": int(attribute["timestamp"]),
                        "Event Date": event["date"],
                        "Event Published": int(event["publish_timestamp"]),
                        "Published": event["published"],
                        "Deleted": attribute["deleted"],
                        "Value": attribute["value"],
                        "Comment": attribute["comment"],
                    }
                )

            for related_event in event["RelatedEvent"]:
                related_events.append(
                    {
                        "Event ID": int(event["id"]),
                        "Orgc ID": int(event["orgc_id"]),
                        "Org ID": int(event["org_id"]),
                        "Date": event["date"],
                        "Related Event ID": int(related_event["Event"]["id"]),
                        "Orgc RE": int(related_event["Event"]["orgc_id"]),
                        "Date RE": related_event["Event"]["date"],
                    }
                )
            if "Tag" in event:
                for tag in event["Tag"]:

                    tags.append(
                        {
                            "Event ID": int(event["id"]),
                            "Orgc ID": int(event["orgc_id"]),
                            "Org ID": int(event["org_id"]),
                            "Date": event["date"],
                            "Timestamp": int(event["timestamp"]),
                            "Analysis": event["analysis"],
                            "Tag ID": int(tag["id"]),
                            "Tag Name": tag["name"],
                            "Tag Colour": tag["colour"],
                        }
                    )

            if "Galaxy" in event:
                for galaxy in event["Galaxy"]:
                    galaxies.append(
                        {
                            "Event ID": int(event["id"]),
                            "Orgc ID": int(event["orgc_id"]),
                            "Date": event["date"],
                            "Galaxy ID": int(galaxy["id"]),
                            "Galaxy Name": galaxy["name"],
                        }
                    )
#             if "GalaxyCluster" in event:
#                 for galaxycluster in event["GalaxyCluster"]:

#                     galaxyclusters.append(
#                         {
#                             "Event ID": int(event["id"]),
#                             "Orgc ID": int(event["orgc_id"]),
#                             "GalaxyCluster ID": int(galaxycluster["id"]),
#                             "GalaxyCluster Name": galaxycluster["name"],
#                         }
#                     )

# Important: Add a part which update all .csv by exporting Event_modified and by comparing the Timestamp of the last update

In [4]:
Events = pd.DataFrame(events)
Events = Events.drop_duplicates(subset=["UUID"], keep="first")
int_columns = [
    "Event ID",
    "Orgc ID",
    "Org ID",
    "Threat Level ID",
    "Attribute Count",
    "Related Events Count",
    "Timestamp",
    "Distribution",
    "Published Timestamp",
]
Events[int_columns] = Events[int_columns].astype(int)
Events = Events.sort_values(["Event ID"]).reset_index().drop(["index"], axis=1)
Events.to_csv(f"{output_folder}/Events.csv", index=False, header=True)

In [5]:
Tags = pd.DataFrame(tags)
Tags.drop_duplicates(subset=["Event ID", "Tag ID"], keep="first")
Tags["Date"] = pd.to_datetime(Tags["Date"])
Tags["Date"] = Tags["Date"].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
Tags["Date"] = Tags["Date"].apply(
    lambda x: time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))
)
Tags["Delta t"] = Tags["Timestamp"] - Tags["Date"]
Tags = Tags.sort_values(["Event ID"]).reset_index().drop(["index"], axis=1)
Tags.to_csv(f"{output_folder}/Tags.csv", index=False, header=True)

In [6]:
Attributes = pd.DataFrame(attributes)
Attributes.drop_duplicates(subset=["Attribute ID"], keep="first")  # UUID
Attributes.to_csv(
    f"{output_folder}/Attributes.csv", index=False, header=True
)  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [7]:
Related_events = pd.DataFrame(related_events)
Related_events.drop_duplicates(
    subset=["Related Event ID"], keep="first"
)  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Related_events.to_csv(
    f"{output_folder}/Related_Events.csv", index=False, header=True
)  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [8]:
Galaxies = pd.DataFrame(galaxies)
Galaxies.drop_duplicates(subset=["Galaxy ID"], keep="first")
Galaxies.to_csv(f"{output_folder}/Galaxies.csv", index=False, header=True)

In [9]:
# GalaxyClusters = pd.DataFrame(galaxyclusters)
# GalaxyClusters.drop_duplicates(subset=["GalaxyCluster ID"], keep="first")
# GalaxyClusters.to_csv(f"{output_folder}/GalaxyClusters.csv", index=False, header=True)

In [10]:
# Create the .csv based on the last added folder, i.e. the last retrieving
data_folder = "/home/gillard/Bureau/MISP_Analysis/Back-up/WEIS2022/Organisations"
organisation_folder = max(
    [os.path.join(data_folder, d) for d in os.listdir(data_folder)],
    key=os.path.getmtime,
)  # "/home/gillard/Bureau/MISP_Analysis/Back-up/Organisations/2022-02-07"

organisations = []

for path, subdirs, files in os.walk(organisation_folder):
    for file in files:
        if file.startswith("org_") and file.endswith(".json"):
            with open(os.path.join(path, file), "r") as org_file:
                print(os.path.join(path, file), end="\r")
                org = json.load(org_file)["Organisation"]

                organisations.append(
                    {
                        "Org ID": int(org["id"]),
                        "Org Name": org["name"],
                        "Org UUID": org["uuid"],
                        "Nationality": org["nationality"],
                        "Date created": org["date_created"],
                        "Date modified": org["date_modified"],
                        "Sector": org["sector"],
                        "User Count": int(org["user_count"]),
                        "Local": str(org["local"]),
                    }
                )

In [11]:
Organisations = pd.DataFrame(organisations)
int_columns = ["Org ID", "User Count"]
Organisations[int_columns] = Organisations[int_columns].astype(int)
Organisations = (
    Organisations.sort_values(["Org ID"]).reset_index().drop(["index"], axis=1)
)
Organisations.to_csv(f"{output_folder}/Organisations.csv", index=False, header=True)